In [21]:
import json
import csv
import random
from datetime import datetime, timedelta
from typing import List
import os


# 定数
DETECTOR_RANGE = 200.0  # 検知範囲（半径）
WALKER_SPEED = 1.4  # 通行人の移動速度（m/s）
VARIATION_FACTOR = 0.2  # 移動時間のばらつき（20%）ere


# クラス定義
class Detector:
    def __init__(self, id: str, x: float, y: float):
        self.id = id
        self.x = x
        self.y = y


# 設定データの読み込み
def load_detectors(file_path: str) -> List[Detector]:
    """
    JSONファイルから検知器情報をロードし、Detectorクラスのリストを返す
    """
    with open(file_path, "r") as file:
        data = json.load(file)
        return [Detector(**d) for d in data["detectors"]]


def load_payloads(file_path: str) -> dict[str, dict[str, float]]:
    """JSONファイルからペイロード確率分布をロード"""
    with open(file_path, "r") as file:
        data:dict = json.load(file)
        return data["models"]






In [22]:
import math

def generate_routes(detectors: List[Detector], num_walkers: int) -> List[List[str]]:
    """通行人ごとの移動ルートを生成"""
    detector_ids = [d.id for d in detectors]
    return [
        random.sample(detector_ids, k=len(detector_ids)) for _ in range(num_walkers)
    ]


def assign_models_to_walkers(num_walkers: int, models: List[str]) -> List[str]:
    """通行人にランダムな機種モデルを割り当てる
    ここはランダムに割り当てるだけなので実際のデータには使えないため、後で確率モデルを使う
    """
    return [random.choice(models) for _ in range(num_walkers)]



def choose_payload_for_model(model: str, payloads: dict) -> str | None:
    """指定されたモデルの確率分布に基づいてペイロードをランダムに選択"""
    model_payloads: dict[str, float] = payloads[model]
    r = random.random()
    cumulative = 0.0
    for payload, probability in model_payloads.items():
        cumulative += probability
        if r <= cumulative:
            return payload
    return None


def calculate_travel_time(ax, ay, bx, by, speed, variation_factor)-> int:
    """検知器AからBへの移動時間を計算し、距離に基づくランダムなばらつきを追加"""
    # ユークリッド距離を計算
    distance = math.sqrt((bx - ax) ** 2 + (by - ay) ** 2)

    # 基本の移動時間を計算
    travel_time = distance / speed  # 秒単位

    # 距離に基づくランダム変動を加える
    variation = random.uniform(
        -travel_time * variation_factor, travel_time * variation_factor
    )
    final_travel_time = max(0, travel_time + variation)  # 負の時間を防ぐためmax(0)

    return final_travel_time

def sort_csv_by_timestamp(file_path: str) -> None:
    """
    CSVファイルをタイムスタンプ順に並べ替える。

    Parameters:
        file_path (str): ソート対象のCSVファイルのパス。
    """
    # ファイルを読み込む
    with open(file_path, "r") as file:
        reader = csv.reader(file)
        header = next(reader)  # ヘッダーを保持
        rows = sorted(reader, key=lambda row: row[0])  # タイムスタンプ列でソート

    # ソート済みデータを再書き込み
    with open(file_path, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(header)  # ヘッダーを書き戻す
        writer.writerows(rows)  # ソート済みデータを書き戻す


def sort_all_detector_logs(detector_ids: List[str]) -> None:
    """
    すべての検知器ログファイルをタイムスタンプ順にソート。

    Parameters:
        detector_ids (List[str]): 検知器IDのリスト。
    """
    for detector_id in detector_ids:
        file_path = f"result/{detector_id}_log.csv"
        sort_csv_by_timestamp(file_path)


In [23]:
def simulate(
    detectors: List[Detector],
    routes: List[List[str]],
    payloads: dict,
    models: List[str],
) -> None:
    """シミュレーションを実行"""
    # 統一したスタート時刻
    start_time = datetime(2024, 1, 14, 11, 0, 0)  # 開始時間: 午前11時
    detector_data = {detector.id: [] for detector in detectors}

    os.makedirs("result", exist_ok=True)  # resultフォルダを作成

    # 歩行者のルートCSVを初期化
    with open("result/walker_routes.csv", "w", newline="") as walker_file:
        walker_writer = csv.writer(walker_file)
        walker_writer.writerow(["Walker_ID", "Route", "Model"])  # モデル情報を追加

        # 全歩行者の処理
        for walker_id, (route, model) in enumerate(zip(routes, models), start=1):
            # ルートを記録
            route_str = "".join(route)
            walker_writer.writerow([walker_id, route_str, model])

            # 現在の時刻をスタート時刻に設定
            current_time = start_time

            for i, detector_id in enumerate(route):
                current_detector = next(d for d in detectors if d.id == detector_id)

                # パケットを一時リストに保存
                for _ in range(10):
                    random_offset = timedelta(
                        seconds=random.randint(0, 5 * 60)
                    )  # 最大5分のランダム
                    event_time = current_time + random_offset
                    chosen_payload = choose_payload_for_model(model, payloads)
                    # hashed_payload = hash(chosen_payload)
                    hashed_payload = chosen_payload

                    # イベントを記録
                    detector_data[detector_id].append(
                        (
                            event_time,
                            hashed_payload,
                            walker_id,
                            current_detector.x,
                            current_detector.y,
                        )
                    )

                # 次の移動時間を計算
                if i < len(route) - 1:
                    next_detector = next(d for d in detectors if d.id == route[i + 1])
                    travel_time = calculate_travel_time(
                        current_detector.x,
                        current_detector.y,
                        next_detector.x,
                        next_detector.y,
                        WALKER_SPEED,
                        VARIATION_FACTOR,
                    )
                    current_time += timedelta(seconds=travel_time)

    # 検知器ごとのCSVファイルに記録
    for detector_id, events in detector_data.items():
        events.sort(key=lambda e: e[0])  # タイムスタンプでソート
        with open(f"result/{detector_id}_log.csv", "w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(
                ["Timestamp", "Hashed_Payload", "User_ID", "Detector_X", "Detector_Y"]
            )
            writer.writerows(events)


def main():
    # JSONデータをロード
    detectors = load_detectors("./config/detectors.json")
    payloads = load_payloads("./config/payloads.json")

    # 通行人のルートとモデルを生成
    num_walkers =4
    routes = generate_routes(detectors, num_walkers)
    models = assign_models_to_walkers(num_walkers, list(payloads.keys()))

    # シミュレーションを実行
    simulate(detectors, routes, payloads, models)

    # 検知器ごとのログファイルをタイムスタンプ順にソート
    detector_ids = [detector.id for detector in detectors]
    sort_all_detector_logs(detector_ids)


if __name__ == "__main__":
    main()

In [24]:
# import random

# distribution = {
#     "iPhone SE": 21.8,
#     "iPhone 13": 16.5,
#     "iPhone 14": 15.1,
#     "iPhone 15": 14.0,
#     "iPhone 12": 13.9,
#     "iPhone 11": 7.3,
#     "iPhone 8": 3.7,
#     "iPhone XR": 2.7,
#     "iPhone 7以前": 2.3,
#     "iPhone XS": 1.5,
#     "AQUOS": 26.4,
#     "Xperia": 19.8,
#     "Galaxy": 14.2,
#     "Google Pixel": 11.8,
#     "OPPO": 7.7,
#     "arrows": 5.4,
#     "Xiaomi": 3.3,
#     "楽天モバイルオリジナル": 1.5,
#     "moto": 1.4,
#     "HUAWEI": 1.4,
#     "Android One": 1.0,
#     "BASIO": 0.8,
#     "ZTE": 0.7,
# }

# def get_smartphone_by_distribution():
#     rand_value = random.uniform(0, sum(distribution.values()))
#     cumulative = 0
#     for model, percentage in distribution.items():
#         cumulative += percentage
#         if rand_value <= cumulative:
#             return model

# # 使用例
# print(get_smartphone_by_distribution())
